<a href="https://colab.research.google.com/github/ileniagalati/computer_vision_project/blob/main/FINALE_TestNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cable and towel detection and segmentation

In [1]:
import locale
import json
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
# Collegamento al drive per ottenere il file json con i risultati del training e il test set, in alternativa inserire di seguito i path corretti
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
test_path = '/content/drive/MyDrive/dataset_progetto_cv/testset'
test_file =  'test.json'
output_path= '/content/drive/MyDrive/output/coco_instances_results.json'

In [4]:
!pip install pycocotools

# Generate output json

In [5]:
#ci serve avere il risultato su due file separati per bbox e maschere
import pycocotools
# Carica le predizioni dal file
with open(output_path, 'r') as file:
    predizioni = json.load(file)

# Estrai box e maschere
box_predizioni = []
maschere_predizioni = []

for predizione in predizioni:
    # Estrai le informazioni necessarie per box e maschere
    image_id= predizione['image_id']
    category_id= predizione['category_id']
    bbox= predizione['bbox']
    score= predizione['score']
    maschera = predizione['segmentation']

    bbox = [round(float(x) * 10) / 10 for x in bbox]

    box_predizioni.append({
        'image_id': int(image_id),
        'category_id': int(category_id),
        'bbox': bbox,
        'score': float(score)
      })

    maschere_predizioni.append({
        'image_id': int(image_id),
        'category_id': int(category_id),
        'segmentation': {
            'size': maschera['size'],
            'counts': maschera['counts']
        },
            'score': float(score)
        })

# Salva le predizioni di box in un nuovo file
with open('bbox_detections.json', 'w') as file:
    json.dump(box_predizioni, file)

# Salva le predizioni di maschere in un nuovo file
with open('mask_detections.json', 'w') as file:
    json.dump(maschere_predizioni, file)


# Compute metrics

In [6]:
test = test_path+'/'+test_file

gt = COCO(test)

gt.cats

loading annotations into memory...
Done (t=3.88s)
creating index...
index created!


{0: {'supercategory': 'cable', 'id': 0, 'name': 'cable'},
 1: {'supercategory': 'tower_lattice', 'id': 1, 'name': 'tower_lattice'},
 2: {'supercategory': 'tower_tucohy', 'id': 2, 'name': 'tower_tucohy'},
 3: {'supercategory': 'tower_wooden', 'id': 3, 'name': 'tower_wooden'}}

In [7]:
gt = COCO(test)
detections = gt.loadRes('/content/bbox_detections.json')

detections.cats

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


{0: {'supercategory': 'cable', 'id': 0, 'name': 'cable'},
 1: {'supercategory': 'tower_lattice', 'id': 1, 'name': 'tower_lattice'},
 2: {'supercategory': 'tower_tucohy', 'id': 2, 'name': 'tower_tucohy'},
 3: {'supercategory': 'tower_wooden', 'id': 3, 'name': 'tower_wooden'}}

In [8]:
# running evaluation
annType = 'bbox'
cocoEval = COCOeval(gt, detections, annType)
#cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.15s).
Accumulating evaluation results...
DONE (t=0.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.433
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.595
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.488
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.495
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.344
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.496
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.507
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.212
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [9]:
# running evaluation
annType = 'segm'
detections = gt.loadRes('/content/mask_detections.json')
cocoEval = COCOeval(gt, detections, annType)
#cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.35s).
Accumulating evaluation results...
DONE (t=0.12s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.226
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.405
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.216
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.021
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.393
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.227
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.299
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.301
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10

In [10]:
with open('/content/bbox_detections.json') as fp_src:
    data = json.load(fp_src)

print(type(data), len(data))
data[0]

<class 'list'> 2855


{'image_id': 0,
 'category_id': 0,
 'bbox': [320.9, 5.1, 76.1, 688.8],
 'score': 0.9997333884239197}

In [13]:
with open('/content/mask_detections.json') as fp_src:
    data = json.load(fp_src)

print(type(data), len(data))
data[0]

<class 'list'> 2855


{'image_id': 0,
 'category_id': 0,
 'segmentation': {'size': [700, 700],
  'counts': 'P[d77de05Mb0^O76F27H49NE6NN42H601K5I52L05I25LN5LO61H32M24I07OM0ON43K21O20L6L40G2J9E8H<^Oh_^6'},
 'score': 0.9997333884239197}